<a href="https://colab.research.google.com/github/ckdghks/colab-AI/blob/master/Keras(3DGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#라이브러리


In [ ]:
import glob
import os
import time

import numpy as np
import scipy.io as io
import scipy.ndimage as nd
import tensorflow as tf
from keras import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv3D, Deconv3D
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt

#생성기 신경망


In [ ]:
def build_generator() :
    """
    다음에 정의한 하이퍼파라미터 값들을 사용하는 생성기 모델을 작성한다.
    :return: 생성기 신경망
    """
    z_size = 200
    gen_filters = [512, 256, 128, 64, 1]
    gen_kernel_sizes = [4, 4, 4, 4, 4]
    gen_strides = [1, 2, 2, 2, 2]
    gen_input_shape = (1, 1, 1, z_size)
    gen_activations = ['relu', 'relu', 'relu', 'relu', 'sigmoid']
    gen_convolutional_blocks = 5

    input_layer = Input(shape=gen_input_shape)

    # 첫 번째 3차원 전치합성곱(즉, 3차원 역합성곱) 블록
    a = Deconv3D(filters=gen_filters[0],
                 kernel_size=gen_kernel_sizes[0],
                 strides=gen_strides[0])(input_layer)
    a = BatchNormalization()(a, training=True)
    a = Activation(activation='relu')(a)

    # 그 다음에 이어서 나오는 네 개의 3차원 전치핪어곱 블록록
    for i in range(gen_convolutional_blocks - 1):
        a = Deconv3D(filters=gen_filters[i + 1],
                     kernel_size=gen_kernel_sizes[i + 1],
                     strides=gen_strides[i + 1], padding='same')(a)
        a = BatchNormalization()(a, training=True)
        a = Activation(activation=gen_activations[i + 1])(a)

    gen_model = Model(inputs=[input_layer], outputs=[a])
    return gen_model

#판별기 신경망

In [ ]:
def build_discriminator() :
    """
    다음과 같이 정의된 하이퍼파라미터 값들을 사용해 판별기 모델을 만든다.
    """

    dis_input_shape = (64, 64, 64, 1)
    dis_filters = [64, 128, 256, 512, 1]
    dis_kernel_sizes = [4, 4, 4, 4, 4]
    dis_strides = [2, 2, 2, 2, 1]
    dis_paddings = ['same', 'same', 'same', 'same', 'valid']
    dis_alphas = [0.2, 0.2, 0.2, 0.2, 0.2]
    dis_activiations = ['leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu', 'sigmoid']
    dis_convolutional_blocks = 5

    dis_input_layer = Inpuut(shape=dis_input_shape)

    # 첫 번째 3차원 합성곱 블록
    a = Conv3D(filters=dis_filters[0]),
                        kernel_size=dis_kernel_sizes[0],
                        strides=dis_strides[0],
                        padding=dis_paddings[0])(dis_input_layer)
    a = BatchNormalization()(a, training=True)
    a = LeakyReLU(dis_alphas[0])(0)

    # 다음으로 추가할 3차원 합성곱 블록 네 개
    for i in range(dis_convolutional_blocks - 1):
        a = Conv3D(filters=dis_filters[i+1],
                            kernel_size=dis_kernel_sizes[i+1],
                   strides=dis_strides[i+1],
                   padding=dis_paddings[i+1])(a)
        a = BatchNormalization()(a, training=True)
        if dis_activiations[i+1] == 'leaky_relu':
            a = LeakyReLU(dis_alphas[i+1])(a)
        elif dis_activiations[i+1] == 'sigmoid':
            a = Activation(activaiton='sigmoid')(a)
        
    dis_model = Model(inputs=dis_input_layer, outputs=a)
    print(dis_model.summary())

    return dis_model

#3D-GAN 훈련

##신경망 훈련

In [ ]:
gen_learnig_rater = 0.0025
dis_learning_rate = 0.00001
beta = 0.5
batch_size = 32
z_size = 200
DIR_PATH = '3DShapenets 데이터셋이 들어있는 디렉터리에 대한 경로'
generated_volumes_dir = 'generated_volumes'
log_dir = 'logs'


In [ ]:
# 인스턴스들을 생성한다.
generator = build_generator()
discriminator = build_discriminator()


In [ ]:
# 최적화기를 지정한다.
gen_optimizer = Adam(lr=gen_learning_rate, beta_1=beta)
dis_optimizer = Adam(lr=dis_learning_rate, beta_1=0.9)


In [ ]:
# 신경망들을 컴파일한다.
generator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.compile(loss='binary_crossentropy', optimizer=dis_optimizer)


In [ ]:
discriminator.trainable = False
adversarial_model = Sequential()
adversarial_model.add(generator)
adversarial_model.add(discriminator)
adversarial_model.compile(loss="binary_crossentropy",
                          optimizer=Adam(lr=gen_learning_rate, beta_1=beta))


In [ ]:
def getVoxelsFromMat(path, cube_len=64):
    voxels = io.loadmat(path)['istance']
    voxels = np.pad(voxels, (1, 1), 'constant', constant_values=(0, 0))
    if cube_len != 32 and cube_len == 64:
        voxels = nd.zoom(voxels, (2, 2, 2), mode='constant', order=0)
    return voxels

In [ ]:
def get3ImagesForACategory(obj='airplane', train=True, cube_len=64, obj_ratio=1.0):
    obj_path = DIR_PATH + obj + '/30/'
    obj_path += 'train/' if train else 'test/'
    fileList = [f for f in os.listdir(obj_path) if f.endswith('.mat')]
    fileList = fileList[0:int(obj_ratio * len(fileLIst))]
    volumeBatch = np.asarray([getVoxelsFromMat(obj_path + f, cube_len) for f in fileList],
                             dtype=np.bool)
    return volumeBatch
    

In [ ]:
volumes = get3ImagesForACategory(obj='airplane', train=True, obj_ratio=1.0)
volumes = volumes[..., np.newaxis].astype(np.float)

In [ ]:
tensorboard = TensorBoard(log_dir="{}/{}".format(log_dir, time.time()))
tensorboard.set_model(generator)
tensorboard.set_model(discriminator)

In [ ]:
for epoch in range(epochs):
    print("Epoch:", epoch)

    # 손실들을 저장할 두 개의 리스트를 만든다.
    gen_losses = []
    dis_losses = []
    

In [ ]:
number_of_batches = int(volumes.shape[0] / batch_size)
print("Number of batches:", number_of_batches)
for index in range(numbr_of_batches):
    print("Batch:", index + 1)


In [ ]:
z_sample = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)
volumes_batch = volumes[index * batch_size:(index + 1) * batch_size, :, :, :]

In [ ]:
gen_volumes = generator.predict(z_sample, verbose=3)


In [ ]:
# 판별기 신경망을 훈련 가능하게 설정한다.
discriminator.trainable = True

# 가짜 레이블과 진짜 레이블을 생성한다.
labels_real = np.reshpae([1] * batch_size, (-1, 1, 1, 1, 1))
labels_fake = np.reshape([0] * batch_size, (-1, 1, 1, 1, 1))

# 생성기 신경망을 훈련한다.
loss_real = discriminator.train_on_batch(volumes_batch, labels_real)
loss_fake = discriminator.train_on_batch(gen_volumes, labels_fake)

# 총 판별기 손실을 계산한다.
d_loss = 0.5 * (loss_real + loss_fake)


In [ ]:
z = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)

# 적대 모델을 훈련한다.
g_loss = adversarial_model.train_on_batch(z, np.reshape([1]* batch_size, (-1, 1, 1, 1, 1)))

gen_losses.append(g_loss)
dis_losses.append(d_loss)


In [ ]:
if index % 10 = 0:
    z_sample2 = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)
    generated_volumes = generator.predict(z_sample2, verbose=3)
for i, generated_volume in enumerate(generated_volumes[:5]):
    voxels = np.squeese(generated_volume)
    voxels[voxels < 0.5] = 0.
    voxels[voxels >= 0.5] = 1.
    saveFromVoxels(voxels, "results/img_{}_{}_{}".format(epoch, index, i))

In [ ]:
# 텐서보드에 손실들을 저장한다.
write_log(tensorboard, 'g_loss', np.mean(gen_losses), epoch)
write_log(tensorboard, 'd_loss', np.mean(dis_losses), epoch)